In [4]:
from fishsense_api_sdk.client import Client
from tqdm.asyncio import tqdm_asyncio
from fishsense_api_sdk.models.species_label import SpeciesLabel
from fishsense_api_sdk.models.dive_frame_cluster import DiveFrameCluster
from datetime import datetime, timezone
from tqdm.notebook import tqdm
from fishsense_data_processing_workflow_worker.config import settings
from fishsense_api_sdk.models.data_source import DataSource

In [2]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    dives = await client.dives.get()
    dives = [dive for dive in dives if dive.priority == "HIGH"]

len(dives), dives[0]

(8,
 Dive(id=383, name='062624_FiveSeas_FSL03', path='2025-02-10 REEF Data Dump SMILE 6/CubaTrip2024_FSL03/062624_FiveSeas_FSL03', dive_datetime=datetime.datetime(2024, 6, 26, 7, 52, 45, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=3, dive_slate_id=None))

In [5]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    dive_image_groups_list = await tqdm_asyncio.gather(*[client.images.get_clusters(dive.id, DataSource.PREDICTION.value) for dive in dives])

len(dive_image_groups_list), len(dive_image_groups_list[0]), dive_image_groups_list[0][0]
    

100%|██████████| 8/8 [00:03<00:00,  2.51it/s]


(8,
 13,
 DiveFrameCluster(id=3044, image_ids=[111930, 111930, 111930, 111930, 111930, 111930, 111930, 111930, 111930, 111930, 111930, 111930, 111930, 111931, 111931, 111931, 111931, 111931, 111931, 111931, 111931, 111931, 111931, 111931, 111931, 111931, 111932, 111932, 111932, 111932, 111932, 111932, 111932, 111932, 111932, 111932, 111932, 111932, 111932, 111933, 111933, 111933, 111933, 111933, 111933, 111933, 111933, 111933, 111933, 111933, 111933, 111933, 111934, 111934, 111934, 111934, 111934, 111934, 111934, 111934, 111934, 111934, 111934, 111934, 111934], data_source=<DataSource.PREDICTION: 'PREDICTION'>, updated_at=datetime.datetime(2025, 12, 8, 20, 59, 30, 781, tzinfo=TzInfo(0)), dive_id=383, fish_id=None))

In [6]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    species_labels = await tqdm_asyncio.gather(*[client.labels.get_species_labels(dive.id) for dive in dives])
    species_labels = [label for sublist in species_labels for label in sublist]

len(species_labels), species_labels[0]

100%|██████████| 8/8 [00:01<00:00,  5.00it/s]


(1218,
 SpeciesLabel(id=1179, label_studio_task_id=224623, label_studio_project_id=70, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/708d385dc06f8fd60c46683478e77b2b', updated_at=datetime.datetime(2025, 12, 9, 1, 2, 5, 406207, tzinfo=TzInfo(0)), completed=True, grouping=None, top_three_photos_of_group=None, slate_upside_down=None, laser_x=2043.1891467172798, laser_y=1355.6551899834365, laser_label='Red Laser', content_of_image='Slate, Laser on slate', fish_measurable_category=None, fish_angle_category=None, fish_curved_category=None, label_studio_json={'annotations': [{'id': 134871, 'result': [{'id': 'X-QEwfPU-K', 'type': 'taxonomy', 'value': {'taxonomy': [['Slate', 'Laser on slate']]}, 'origin': 'manual', 'to_name': 'image', 'from_name': 'species'}, {'id': 'w_p1HuzBVT', 'type': 'keypointlabels', 'value': {'x': 50.901573161865464, 'y': 44.94877950873463, 'width': 0.2762390456930156, 'keypointlabels': ['Red Laser']}, 'origin': 'manual', 'to_name': 'image

In [7]:
species_label_by_image_id = {label.image_id: label for label in species_labels}

len(species_label_by_image_id)

1218

In [8]:
grouped_species_labels = [[[(group.dive_id, species_label_by_image_id[idx]) for idx in group.image_ids] for group in dive] for dive in dive_image_groups_list]

len(grouped_species_labels), len(grouped_species_labels[0]), len(grouped_species_labels[0][0]), grouped_species_labels[0][1][0:5]

(8,
 13,
 65,
 [(383,
   SpeciesLabel(id=1205, label_studio_task_id=224649, label_studio_project_id=70, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/b79fed084d5817f81b37aaa4164f1194', updated_at=datetime.datetime(2025, 12, 9, 1, 17, 29, 655924, tzinfo=TzInfo(0)), completed=True, grouping=None, top_three_photos_of_group=None, slate_upside_down=None, laser_x=None, laser_y=None, laser_label=None, content_of_image='Fish, Other (Identifiable but Nontarget)', fish_measurable_category='yes, center of fish', fish_angle_category=None, fish_curved_category=None, label_studio_json={'annotations': [{'id': 134898, 'result': [{'id': 'XPb8z957rQ', 'type': 'taxonomy', 'value': {'taxonomy': [['Fish', 'Other (Identifiable but Nontarget)']]}, 'origin': 'manual', 'to_name': 'image', 'from_name': 'species'}, {'id': 'ETPOSS0HCV', 'type': 'taxonomy', 'value': {'taxonomy': [['yes, center of fish']]}, 'origin': 'manual', 'to_name': 'image', 'from_name': 'measurable'}], 'create

In [10]:
dive_groups = []

current_group = []
for dive_label_group in grouped_species_labels:
    dive = []

    for label_group in dive_label_group:
        for idx, (dive_id, label) in enumerate(label_group):
            if (idx == 0 and label.grouping != "Part of previous group") or label.grouping == "Not part of current group":
                if current_group:
                    dive.append(current_group)
                current_group = []

            current_group.append((dive_id, label))

    if current_group:
        dive.append(current_group)
        current_group = []

    if dive:
        dive_groups.append(dive)
        dive = []

len(dive_groups), len(dive_groups[0]), dive_groups[0]

(8,
 39,
 [[(383,
    SpeciesLabel(id=1203, label_studio_task_id=224644, label_studio_project_id=70, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/c5cef715ad648236aac9c06f762d2ead', updated_at=datetime.datetime(2025, 12, 9, 1, 15, 11, 765996, tzinfo=TzInfo(0)), completed=True, grouping=None, top_three_photos_of_group=None, slate_upside_down=None, laser_x=None, laser_y=None, laser_label=None, content_of_image=None, fish_measurable_category=None, fish_angle_category=None, fish_curved_category=None, label_studio_json={'annotations': [{'id': 134893, 'result': [], 'created_username': ' alli@reef.org, 10', 'created_ago': '1\xa0hour, 48\xa0minutes', 'completed_by': 10, 'was_cancelled': False, 'ground_truth': False, 'created_at': '2025-12-09T01:15:11.765983Z', 'updated_at': '2025-12-09T01:15:11.765996Z', 'draft_created_at': None, 'lead_time': 26.865, 'import_id': None, 'last_action': None, 'bulk_created': False, 'task': 224644, 'project': 70, 'updated_by': 10, 

In [11]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    for dive in tqdm(dive_groups):
        for grouping in dive:
            dive_id = grouping[0][0]

            cluster = DiveFrameCluster(
                id = None,
                dive_id=dive_id,
                image_ids=[label.image_id for _, label in grouping],
                data_source="LABEL_STUDIO",
                fish_id=None,
                updated_at=datetime.now(timezone.utc)
            )
                
            await client.images.post_cluster(dive_id, cluster)

  0%|          | 0/8 [00:00<?, ?it/s]